In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers, metrics

In [95]:
#del mnist_dataset
def prepare_mnist_features_and_labels(x,y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x,y

def mnist_dataset():
    (x,y),(x_val,y_val) = datasets.fashion_mnist.load_data()
    print("x.shape",x.shape)
    print("y.shape",y.shape)
    y = tf.one_hot(y, depth=10)
    y_val = tf.one_hot(y_val, depth=10)
    ds = tf.data.Dataset.from_tensor_slices((x,y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.shuffle(60000).batch(100)
    
    ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    ds_val = ds_val.map(prepare_mnist_features_and_labels)
    ds_val = ds_val.shuffle(60000).batch(100)
    
    return ds, ds_val

ds_train, ds_val = mnist_dataset()

In [67]:

def create_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(28,28), name="inputs"))
    model.add(layers.Reshape(target_shape=(28*28,), name="reshape_1"))
    model.add(layers.Dense(units=200, activation="relu", name="hidden_1"))
    model.add(layers.Dense(units=200, activation="relu", name="hidden_2"))
    model.add(layers.Dense(units=200, activation="relu", name="hidden_3"))
    model.add(layers.Dense(units=10, activation="softmax", name="outputs"))
    model.summary()
    
    model.compile(loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                  optimizer=optimizers.Adam(lr=0.05),
                  metrics=["accuracy"])
    return model

model = create_model()
# WARNING:tensorflow:Your input ran out of data; interrupting training. 
# Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches 
# (in this case, 15000 batches). You may need to use the repeat() function when building your dataset.

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 784)               0         
_________________________________________________________________
hidden_1 (Dense)             (None, 200)               157000    
_________________________________________________________________
hidden_2 (Dense)             (None, 200)               40200     
_________________________________________________________________
hidden_3 (Dense)             (None, 200)               40200     
_________________________________________________________________
outputs (Dense)              (None, 10)                2010      
Total params: 239,410
Trainable params: 239,410
Non-trainable params: 0
_________________________________________________________________


In [49]:
history = model.fit(ds_train.repeat(), epochs=3, steps_per_epoch=500,
                    validation_data=ds_val.repeat(), validation_steps=2, verbose=1)

Train for 500 steps, validate for 2 steps
Epoch 1/3
500/500 [==============================] - 29s 57ms/step - loss: 2.3613 - accuracy: 0.0998 - val_loss: 2.3262 - val_accuracy: 0.1350
Epoch 2/3
500/500 [==============================] - 29s 58ms/step - loss: 2.3613 - accuracy: 0.0998 - val_loss: 2.3562 - val_accuracy: 0.1050
Epoch 3/3
500/500 [==============================] - 34s 68ms/step - loss: 2.3620 - accuracy: 0.0991 - val_loss: 2.3362 - val_accuracy: 0.1250


In [50]:
print(history.history.keys())
loss = [_loss for _loss in history.history['loss'][:]]
size = len(loss)
print("loss:\t", end="")
for i in range(size):
    print("%.4f"%loss[i], " ",end="")
print()
acc = [_acc for _acc in history.history['accuracy'][:]]
print("acc:\t", end="")
for i in range(size):
    print("%.4f"%acc[i], " ", end="")
print()
v_loss = [_v_loss for _v_loss in history.history['val_loss'][:]]
print("v_loss:\t", end="")
for i in range(size):
    print("%.4f"%v_loss[i], " ", end="")
print()
v_acc = [_v_acc for _v_acc in history.history['val_accuracy'][:]]
print("v_acc:\t", end="")
for i in range(size):
    print("%.4f"%v_acc[i], " ", end="")
print()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
loss:	2.3613  2.3613  2.3620  
acc:	0.0998  0.0998  0.0991  
v_loss:	2.3262  2.3562  2.3362  
v_acc:	0.1350  0.1050  0.1250  


In [40]:
# accuracy 1%，不預測

## Customer Layer

In [92]:
#　model = MyLayer([28*28, 200, 200, 10])
#del MyLayer
class MyLayer(layers.Layer):
    def __init__(self, units):
        super(MyLayer, self).__init__()
        for i in range(1, len(units)):
            # w: [input_dim, output_dim]
            self.add_variable(name="kernel%d"%i, shape=[units[i-1], units[i]])
            # b: [output_dim]
            self.add_variable(name="bias%d"%i, shape=[units[i]])
            # layer 1 [784,200] [200]
            # layer 2 [200,200] [200]
            # layer 3 [200,10] [10]            
    def call(self, x):
        num = len(self.trainable_variables)
        #print("len(trainable_variables)",num) output 6 => 3layer weight+bias
        x = tf.reshape(x, [-1,28*28])
        for i in range(0, num, 2):
            x = tf.matmul(x, self.trainable_variables[i]) + self.trainable_variables[i+1]
            #print(self.trainable_variables[i].name)
        return x
# Please use `layer.add_weight` method instead add_variable

In [98]:
#del mnist_dataset
def mnist_dataset():
    (x, y), _ = datasets.fashion_mnist.load_data()

    print('x/y shape:', x.shape, y.shape)

    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds
train_dataset = mnist_dataset()

x/y shape: (60000, 28, 28) (60000,)


In [96]:
#del model
model = MyLayer([28*28, 200, 200, 10])
optimizer = optimizers.Adam()

x.shape (60000, 28, 28)
y.shape (60000,)


In [83]:
for p in model.trainable_variables:
    print(p.name, p.shape)

kernel1:0 (784, 200)
bias1:0 (200,)
kernel2:0 (200, 200)
bias2:0 (200,)
kernel3:0 (200, 10)
bias3:0 (10,)


In [99]:
def compute_loss(logits, labels):
    return tf.reduce_mean(
      tf.nn.sparse_softmax_cross_entropy_with_logits(
          logits=logits, labels=labels))
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy
def train(train_ds):
    #train_ds, val_ds = mnist_dataset()
    loss = 0.0
    accuracy = 0.0    
    for step, (x,y) in enumerate(train_ds):
        loss, accuracy = train_one_step(model, optimizer, x, y)
        if (step+1)%100 == 0:
            print(': loss', loss.numpy(), '; accuracy', accuracy.numpy())
    print("finish training")
    return loss, accuracy

train(train_dataset)

: loss 0.5796629 ; accuracy 0.79
: loss 0.41130623 ; accuracy 0.83
finish training


(<tf.Tensor: id=102111, shape=(), dtype=float32, numpy=0.41130623>,
 <tf.Tensor: id=102178, shape=(), dtype=float32, numpy=0.83>)

## Customer Model

In [60]:
#del mnist_dataset
def mnist_dataset():
    (x, y), (x_val, y_val) = datasets.fashion_mnist.load_data()
    print('x/y shape:', x.shape, y.shape)
    y = tf.one_hot(y, depth=10)
    y_val = tf.one_hot(y_val, depth=10)

    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.shuffle(60000).batch(100)

    ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    ds_val = ds_val.map(prepare_mnist_features_and_labels)
    ds_val = ds_val.shuffle(10000).batch(100)

    sample = next(iter(ds))
    print('sample:', sample[0].shape, sample[1].shape)

    return ds,ds_val

train_dataset, val_dataset = mnist_dataset()

x/y shape: (60000, 28, 28) (60000,)
sample: (100, 28, 28) (100, 10)


In [64]:
#del MyModel
class MyModel(models.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = layers.Dense(units=200, activation="relu", name="hidden_1")
        self.layer2 = layers.Dense(units=200, activation="relu", name="hidden_1")
        self.layer3 = layers.Dense(units=10, activation="softmax", name="outputs")
    
    def call(self, x, training=False):
        x = tf.reshape(x, [-1,28*28])
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        return x
#del model
model = MyModel()

model.compile(loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=optimizers.Adam(1e-3),
              metrics=["accuracy"])

history = model.fit(train_dataset.repeat(), epochs=3, steps_per_epoch=500, verbose=1,
              validation_data=val_dataset.repeat(),
              validation_steps=2)

#model.summary()

Train for 500 steps, validate for 2 steps
Epoch 1/3
500/500 [==============================] - 25s 51ms/step - loss: 1.7450 - accuracy: 0.7239 - val_loss: 1.6676 - val_accuracy: 0.8000
Epoch 2/3
500/500 [==============================] - 20s 39ms/step - loss: 1.6753 - accuracy: 0.7864 - val_loss: 1.6818 - val_accuracy: 0.7750
Epoch 3/3
500/500 [==============================] - 21s 41ms/step - loss: 1.6722 - accuracy: 0.7896 - val_loss: 1.6095 - val_accuracy: 0.8550
